In [ ]:
!pip install accelerate

In [5]:
import os


os.environ["HF_TOKEN"] = 'token_goes_here'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
)

model.to(device)

print(model.device)

In [ ]:
messages = [
    {"role": "system", "content": "Você é um especialista em Matemática."},
    {"role": "user", "content": """Iniciamos nossa jornada com uma visão abrangente da teoria dos números, começando com os conceitos básicos de números primos e suas propriedades. Examinamos a importância do Teorema Fundamental da Aritmética e exploramos como ele garante a fatoração única em primos, essencial para o desenvolvimento subsequente de algoritmos criptográficos.
Avançamos para a análise combinatória, discutindo as fórmulas de combinação e permutação e suas aplicações em problemas de contagem e probabilidade. Demonstramos como esses princípios são utilizados para resolver problemas complexos de contagem em cenários de torneios e na teoria dos grafos.
Um dos aspectos mais fascinantes é a aplicação desses conceitos em sistemas criptográficos, como o RSA, que depende profundamente do entendimento de primos e de sua inacessibilidade através de métodos de fatoração tradicionais. Exploramos também a aplicação de técnicas de combinação em algoritmos de criptografia para criar chaves mais robustas.
Agora, consideremos um problema específico que envolve a aplicação combinada de teoria dos números e análise combinatória:
Problema: Dada uma sequência complexa de operações envolvendo números primos e arranjos, onde cada número primo abaixo de 100 deve ser combinado com outros para formar estruturas de dados únicas que seguem regras específicas de formação, determine quantas estruturas distintas podem ser formadas. Essas estruturas devem ser capazes de se rearranjar para formar novas configurações sem repetir as combinações iniciais. Considere a implicação da mudança de um único elemento na configuração geral.
Este problema não apenas testa sua compreensão dos conceitos matemáticos, mas também sua habilidade em aplicar esses conceitos em cenários altamente complexos e interconectados. Qual seria sua abordagem para começar a resolver esse problema?"""},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

In [ ]:
!nvidia-smi